<img style="float: left;" src="http://third.datacastle.cn/pkbigdata/master.other.img/8ef429f9-2032-47b8-bcc4-818fa9e41a25.png" width="50%">

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
op_train = pd.read_csv('../data/operation_train_new.csv')
tran_train = pd.read_csv('../data/transaction_train_new.csv')
tag_train = pd.read_csv('../data/tag_train_new.csv')

In [3]:
print(op_train.shape)
print(tran_train.shape)

(1460843, 20)
(264654, 27)


### 1.数据探索

In [4]:
def find_different_uid(op_uids, tran_uids):
    op_diff_uids = []
    for uid in op_uids:
        if uid not in tran_uids:
            op_diff_uids.append(uid)
        else:
            tran_uids.remove(uid)
    return op_diff_uids, tran_uids

def find_same_uid(src1_uids, src2_uids):
    same_uids = [uid for uid in src1_uids if uid in src2_uids]
    return same_uids

In [5]:
# 处理时间字符串
op_train['time'] = op_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + op_train['time']
op_train['timestamp'] = op_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

tran_train['time'] = tran_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + tran_train['time']
tran_train['timestamp'] = tran_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

In [6]:
# 根据UID进行排序 再根据timestamp进行排序 
op_train = op_train.sort_values(by=['UID', 'timestamp'],ascending=True)
tran_train = tran_train.sort_values(by=['UID', 'timestamp'],ascending=True)
tag_train = tag_train.sort_values(by=['UID'], ascending=True)
op_train_gb = op_train.groupby('UID', as_index=False)
tran_train_gb = tran_train.groupby('UID', as_index=False)

#### 1.1对op进行分析

In [7]:
#  获取op和tran各自的uid
op_train_uids = [uid for uid, item in op_train_gb.groups.items()]
tran_train_uids = [uid for uid, item in tran_train_gb.groups.items()]

# 获取op和tran各自的tag
op_tag = tag_train[tag_train['UID'].isin(op_train['UID'])]
tran_tag = tag_train[tag_train['UID'].isin(tran_train['UID'])]

In [8]:
# 将op数据中的作假者与正常人分离开
op_train_gb_cnt = op_train_gb.count()
op_train_fake_stat = op_train_gb_cnt[op_train_gb_cnt.index.isin(op_tag['UID'][op_tag['Tag'] == 1])]
op_train_normal_stat = op_train_gb_cnt[op_train_gb_cnt.index.isin(op_tag['UID'][op_tag['Tag'] == 0])]

op_train_fake_gp = op_train[op_train['UID'].isin(op_tag['UID'][op_tag['Tag'] == 1])].groupby('UID', as_index=False)
op_train_normal_gp = op_train[op_train['UID'].isin(op_tag['UID'][op_tag['Tag'] == 0])].groupby('UID', as_index=False)

In [9]:
print(len(op_train_fake_gp))
op_train_fake_gp.head()

4277


,UID,day,mode,success,time,os,version,device1,device2,device_code1,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
87507,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
307385,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
420183,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
216744,10000,13,270ca3ca8f0d1126,1.0,2018-08-13 14:22:59,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
96940,10000,26,c8741ce15ceac2a4,1.0,2018-08-26 12:23:48,103,7.0.7,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09
266255,10019,1,c8741ce15ceac2a4,1.0,2018-08-01 21:58:47,103,7.0.5,aca4977fbe8741e1,IPHONE 5C,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
281109,10019,1,c8741ce15ceac2a4,1.0,2018-08-01 21:58:47,103,7.0.5,aca4977fbe8741e1,IPHONE 5C,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
12591,10019,1,d25caee90b27fa9b,1.0,2018-08-01 21:59:45,103,7.0.5,aca4977fbe8741e1,NaN,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
89575,10019,1,d25caee90b27fa9b,1.0,2018-08-01 22:00:05,103,7.0.5,aca4977fbe8741e1,NaN,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,a395aa3eec2296ca,NaN,NaN,NaN,87f07623623e49e9,NaN,1.533132e+09
226080,10019,6,c8741ce15ceac2a4,1.0,2018-08-06 18:03:36,103,7.0.5,aca4977fbe8741e1,IPHONE 5C,NaN,...,d9cf6be4f1fb9dfa,NaN,NaN,065928d73f7c9597,NaN,NaN,NaN,62bfad343395c711,NaN,1.533550e+09


In [10]:
print(len(op_train_normal_gp))
op_train_normal_gp.head()

25451


,UID,day,mode,success,time,os,version,device1,device2,device_code1,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
214008,10001,2,c8741ce15ceac2a4,1.0,2018-08-02 08:30:52,102,6.6.2,09baf2f39bc3dc86,HUAWEI GRA-CL00,e56819f72c9b7860,...,NaN,NaN,978e38f2d0431a30,2c1958d001a993d9,NaN,NaN,wsgb,bd5e6a16e61e61f7,NaN,1.533170e+09
224571,10001,2,c8741ce15ceac2a4,1.0,2018-08-02 08:30:54,102,6.6.2,09baf2f39bc3dc86,HUAWEI GRA-CL00,e56819f72c9b7860,...,NaN,NaN,978e38f2d0431a30,2c1958d001a993d9,NaN,NaN,wsgb,bd5e6a16e61e61f7,NaN,1.533170e+09
127816,10001,2,79f8b86938180c3c,NaN,2018-08-02 08:31:16,200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533170e+09
364612,10001,2,79f8b86938180c3c,NaN,2018-08-02 08:31:17,200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.533170e+09
28226,10001,2,d25caee90b27fa9b,1.0,2018-08-02 08:31:23,102,6.6.2,09baf2f39bc3dc86,NaN,e56819f72c9b7860,...,NaN,NaN,978e38f2d0431a30,2c1958d001a993d9,NaN,NaN,wsgb,bd5e6a16e61e61f7,NaN,1.533170e+09
238503,10002,29,1c341176507fbd9b,1.0,2018-08-29 09:52:11,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,7b861d5eb7c72dda,...,NaN,NaN,31c72f5ec93149ee,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
351536,10002,29,9c69742a831d6214,1.0,2018-08-29 09:52:11,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,7b861d5eb7c72dda,...,NaN,2af8f4efc0bfdf01,NaN,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
59477,10002,29,8c7ce5ff939e66d1,1.0,2018-08-29 09:52:32,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,7b861d5eb7c72dda,...,NaN,a8dc52f65085212e,NaN,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
198345,10002,29,ac63e881c4e19402,1.0,2018-08-29 09:52:32,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,7b861d5eb7c72dda,...,NaN,a8dc52f65085212e,NaN,7716777f90955a08,NaN,NaN,NaN,4dc6f70900fadcab,NaN,1.535508e+09
31038,10002,29,c8741ce15ceac2a4,1.0,2018-08-29 09:53:10,102,7.0.9,09baf2f39bc3dc86,KIW-AL10,7b861d5eb7c72dda,...,NaN,NaN,31c72f5ec93149ee,7716777f90955a08,NaN,NaN,wmgu,4dc6f70900fadcab,NaN,1.535508e+09


In [50]:
op_feture_columns = ['day', 'mode', 'success', 'time', 'os', 'version',
                     'device1', 'device2', 'device_code1', 'device_code2', 'device_code3',
                     'mac1', 'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
                     'timestamp']

Index(['index', 'UID', 'day', 'mode', 'success', 'time', 'os', 'version',
       'device1', 'device2', 'device_code1', 'device_code2', 'device_code3',
       'mac1', 'mac2', 'ip1', 'ip2', 'wifi', 'geo_code', 'ip1_sub', 'ip2_sub',
       'timestamp'],
      dtype='object')

In [12]:
op_train_fake_gp.get_group(10000)

,UID,day,mode,success,time,os,version,device1,device2,device_code1,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
87507,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
307385,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
420183,10000,13,c8741ce15ceac2a4,1.0,2018-08-13 14:20:56,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
216744,10000,13,270ca3ca8f0d1126,1.0,2018-08-13 14:22:59,103,7.0.5,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,31599eec867ba9c7,NaN,NaN,NaN,ee08d48e73120da4,NaN,1.534141e+09
96940,10000,26,c8741ce15ceac2a4,1.0,2018-08-26 12:23:48,103,7.0.7,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09
168206,10000,26,c8741ce15ceac2a4,1.0,2018-08-26 12:23:48,103,7.0.7,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09
196361,10000,26,c8741ce15ceac2a4,1.0,2018-08-26 12:23:48,103,7.0.7,aca4977fbe8741e1,IPHONE 5,NaN,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09
339411,10000,26,d25caee90b27fa9b,1.0,2018-08-26 12:24:01,103,7.0.7,aca4977fbe8741e1,NaN,NaN,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09
150480,10000,26,d25caee90b27fa9b,1.0,2018-08-26 12:24:21,103,7.0.7,aca4977fbe8741e1,NaN,NaN,...,deb68f7028682008,NaN,NaN,34c715882de1a508,NaN,NaN,NaN,1e3ea9498c461cbf,NaN,1.535257e+09


In [43]:
type(op_train_fake_gp.get_group(10000))
df_10000 = op_train_fake_gp.get_group(10000)
df_10000['mode'].value_counts()

c8741ce15ceac2a4    6
d25caee90b27fa9b    2
270ca3ca8f0d1126    1
Name: mode, dtype: int64

In [50]:
op_train['mode'].value_counts(dropna=False).index

Index(['c8741ce15ceac2a4', 'd25caee90b27fa9b', '79f8b86938180c3c',
       'b501fa4fc58206b9', 'acfaded7e04e7ba0', '072eee5c88d380df',
       '3b02bcd4b685d7a8', '00094ae2a1d62504', '963bd8a75ff9ab37',
       '20a91b45ef8f8221', 'e1d2fad8ffdc66b3', 'cd75f033b0488ace',
       '1c341176507fbd9b', '2f3e878175e34d9c', 'cca82161b59127c1',
       '6b2aa6745680f08a', 'bf79b3647c0878eb', 'd21cae2b73884a8d',
       '479617cd802ed152', '08017d2cb28c2348', 'e806d126013b42d1',
       '09080b31b40d57e8', 'ac63e881c4e19402', '6440f87037199039',
       '37031ef728f5c886', '9c69742a831d6214', 'dd06c9d2ea0a3750',
       '4c3d45ed38eefb7a', '745961ec187394cd', '8c7ce5ff939e66d1',
       '495eeb98eb9b73b8', '270ca3ca8f0d1126', 'b0d4cd119fbef366',
       'fc147276b8ff76fb', '4b55abf363c83025', '33518cc0c2151469',
       'b668e42707ee9c7b', 'cc273b0b2a2afb0f', 'a96a05783c922913',
       'a698c1862f390021', '9037ea11b74dccb9', '56624f0cf0173e89',
       '443ba9b183118540', '4d475374a72edc03', 'd3c998b9bd4684

In [19]:
for key, item in op_train_fake_gp.indices.items():
    print(key, item)

10000 [0 1 2 3 4 5 6 7 8]
10019 [ 9 10 11 12 13 14 15 16 17 18]
10021 [ 19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209]
10041 [210 211 212 213 214 215 216 217]
10058 [218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 24

17590 [34202 34203 34204 34205 34206 34207 34208 34209 34210 34211 34212 34213
 34214 34215 34216 34217 34218 34219 34220 34221 34222 34223 34224 34225
 34226 34227]
17607 [34228 34229 34230 34231 34232 34233 34234 34235 34236 34237 34238 34239
 34240 34241 34242 34243 34244 34245 34246 34247 34248 34249 34250 34251
 34252 34253 34254 34255 34256 34257 34258 34259 34260 34261 34262 34263
 34264 34265 34266 34267 34268 34269 34270 34271 34272 34273 34274 34275
 34276 34277 34278 34279 34280 34281 34282 34283 34284 34285 34286 34287
 34288 34289 34290 34291 34292 34293 34294 34295 34296 34297]
17613 [34298 34299 34300 34301 34302 34303 34304 34305 34306 34307 34308 34309
 34310 34311 34312 34313 34314 34315 34316 34317 34318 34319 34320 34321
 34322 34323 34324 34325 34326 34327 34328 34329 34330 34331 34332 34333
 34334 34335 34336 34337 34338 34339 34340 34341 34342 34343 34344 34345
 34346 34347 34348 34349 34350 34351 34352 34353 34354 34355 34356 34357
 34358]
17615 [34359 34360 343

52096 [72511 72512 72513 72514 72515 72516 72517 72518 72519 72520 72521 72522
 72523 72524 72525 72526 72527 72528 72529 72530 72531 72532 72533 72534
 72535 72536 72537 72538 72539 72540 72541 72542 72543 72544 72545]
52124 [72546 72547 72548 72549 72550 72551 72552]
52126 [72553 72554 72555 72556 72557 72558 72559 72560 72561 72562 72563]
52135 [72564 72565 72566 72567 72568 72569 72570 72571 72572 72573 72574 72575
 72576 72577 72578 72579 72580 72581]
52147 [72582 72583 72584 72585 72586]
52158 [72587 72588 72589 72590 72591 72592 72593 72594 72595 72596 72597]
52160 [72598 72599 72600 72601 72602 72603]
52161 [72604 72605 72606 72607 72608]
52164 [72609 72610 72611 72612 72613 72614 72615 72616 72617 72618 72619 72620
 72621 72622 72623 72624 72625 72626]
52165 [72627 72628 72629 72630 72631 72632 72633]
52167 [72634 72635 72636 72637]
52168 [72638 72639 72640 72641 72642 72643 72644 72645 72646 72647 72648 72649
 72650 72651 72652 72653 72654 72655 72656 72657 72658 72659 72660 

 113937 113938 113939 113940 113941 113942]
59760 [113943 113944 113945 113946 113947 113948 113949 113950 113951 113952
 113953 113954 113955 113956 113957 113958]
59763 [113959 113960 113961 113962 113963 113964 113965 113966 113967 113968
 113969 113970 113971 113972 113973]
59782 [113974 113975 113976 113977 113978 113979 113980 113981 113982 113983
 113984 113985 113986 113987 113988 113989 113990 113991 113992]
59794 [113993 113994 113995 113996 113997 113998 113999 114000 114001 114002
 114003 114004 114005 114006 114007 114008 114009 114010 114011 114012
 114013 114014 114015 114016 114017 114018 114019 114020 114021 114022
 114023 114024 114025 114026 114027 114028 114029 114030 114031 114032
 114033 114034 114035 114036 114037 114038 114039 114040 114041 114042
 114043 114044 114045 114046 114047 114048 114049 114050 114051 114052
 114053 114054 114055 114056 114057 114058 114059 114060 114061 114062
 114063 114064 114065 114066 114067]
59796 [114068 114069 114070 114071 1140

 144521 144522 144523 144524 144525 144526 144527]
65565 [144528 144529 144530 144531 144532 144533 144534 144535 144536 144537
 144538 144539 144540 144541 144542 144543 144544 144545 144546 144547
 144548 144549 144550 144551 144552]
65579 [144553 144554 144555 144556 144557 144558 144559 144560 144561 144562
 144563 144564 144565 144566]
65588 [144567 144568 144569 144570 144571 144572 144573 144574 144575 144576
 144577 144578 144579 144580 144581 144582 144583 144584 144585 144586
 144587 144588 144589 144590 144591 144592 144593 144594 144595 144596
 144597 144598 144599 144600 144601 144602 144603 144604]
65592 [144605 144606 144607 144608 144609 144610 144611 144612 144613 144614
 144615 144616 144617 144618 144619 144620 144621 144622 144623 144624
 144625 144626 144627 144628 144629 144630 144631 144632 144633 144634
 144635 144636 144637 144638 144639 144640 144641 144642 144643 144644]
65597 [144645 144646 144647 144648 144649 144650 144651 144652 144653 144654
 144655 1446